In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import ast

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet as wn

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
from recettes_et_sentiments.api_model import data, preprocessing


In [7]:
my_data = pd.read_parquet("../batch-1672-recettes-et-sentiments-data/basic_preproc_recipes.parquet")
df = my_data[0:8000].copy()

In [6]:
df

,name,minutes,contributor_id,submitted,tags,n_steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps
id,,,,,,,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,minute le course main ingredient cuisine prepa...,11,autumn favorite time year cook recipe prepared...,make choice proceed recipe depending size squa...,7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,make choice proceed recipe depending size squa...
31490,bit different breakfast pizza,30,26278,2002-06-17,minute le course main ingredient cuisine prepa...,9,recipe call crust prebaked bit adding ingredie...,preheat oven degree f press dough bottom side ...,6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,preheat oven degree f press dough bottom side ...
112140,kitchen chili,130,196586,2005-02-25,course preparation main dish chili crock pot s...,6,modified version mom chili hit christmas party...,brown ground beef large pot add chopped onion ...,13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,brown ground beef large pot add chopped onion ...
59389,alouette potato,45,68585,2003-04-14,minute le course main ingredient preparation o...,11,super easy great tasting make ahead side dish ...,place potato large pot lightly salted water br...,11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,place potato large pot lightly salted water br...
44061,amish tomato ketchup canning,190,41706,2002-10-25,weeknight course main ingredient cuisine prepa...,5,dh amish mother raised recipe much prefers sto...,mix ingredient boil hour thick pour jar use ol...,8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,mix ingredient boil hour thick pour jar use ol...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81712,apricot barbecued chicken,45,67899,2004-01-21,minute le course main ingredient preparation o...,6,look scrumptious sweet wait try jan feb issue ...,large skillet cook chicken garlic oil browned ...,8,266.6,7.0,61.0,9.0,53.0,4.0,9.0,large skillet cook chicken garlic oil browned ...
107482,apricot barley casserole,21,52282,2005-01-03,minute le course main ingredient cuisine prepa...,8,easy throw together bakes prepare rest meal es...,preheat oven grease x baking dish large skille...,7,393.7,23.0,51.0,29.0,23.0,23.0,18.0,preheat oven grease x baking dish large skille...
115225,apricot bar,40,68911,2005-04-04,minute le course preparation healthy lunch die...,10,always liked apricot high nutritional value re...,preheat oven oven spray x inch pan set aside p...,8,212.6,18.0,54.0,1.0,10.0,5.0,7.0,preheat oven oven spray x inch pan set aside p...


In [27]:
def get_stopwords(type:int=0)->set:
    """
    Return english (stopwords) from NLTK if type=0

    TODO : support stopwords customized for our use case by passing other value for type

    Args:
        type (int): Type of stopwords to return.
            0 => default english NLTK stopwords
            Other value raise ValueError as of now

    Returns:
        Set[str]: set of stopwords

    Raises:
        ValueError: if type is not supported
    """
    stop_words = set(stopwords.words('english'))

    if type == 0:
        return stop_words

    raise ValueError(f"type {type} not supported")



def remove_stopwords(text:str, stop_words:set)->str:
    """
    Return the "text" argument without the stop_words


    Args:
        text (str): text to be processed.
        stop_words (set) : set of stop words to be used for processing the text.

    Returns:
        str: the input text without stop words
    """
    tokens = word_tokenize(text)
    tokenized_sentence_no_stopwords = [w for w in tokens if not w in stop_words]
    return ' '.join(tokenized_sentence_no_stopwords)


def lemmatize_text(text:str)->str:
    """
    Return the lemmatize "text"

    Args:
        text (str): text to be processed. Text should have been preprocessed (lowercase, ponctuation removed, stopwords removed)

    Returns:
        str: the input text without stop words
    """
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wn.ADJ
        elif treebank_tag.startswith('V'):
            return wn.VERB
        elif treebank_tag.startswith('N'):
            return wn.NOUN
        elif treebank_tag.startswith('R'):
            return wn.ADV
        else:
            return wn.NOUN  # Par défaut, retourne NOM

    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    lemmatizer = WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in pos_tags]

    return ' '.join(lemmatized_tokens)

In [11]:
df.iloc[0]['merged_steps']

'make choice proceed recipe depending size squash cut half fourth remove seed spicy squash drizzle olive oil melted butter cut squash piece season mexican seasoning mix ii sweet squash drizzle melted honey butter grated piloncillo cut squash piece season sweet mexican spice mix bake degree depending size minute hour fork easily pierce skin careful burn squash especially opt use sugar butter feel comfortable cover squash aluminum foil first half hour give take baking desired season salt'

In [28]:
stop_words = get_stopwords()

In [36]:
df['merged_ingredients'] = df['ingredients'].apply(lambda x: "\n".join(x))


In [38]:
df['merged_steps'] = df['steps'].apply(lambda x: "\n".join(x))
df['merged_tags'] = df['tags'].apply(lambda x: "\n".join(x))

In [39]:
df['merged_ingredients_wo_sw'] = df['merged_ingredients'].apply(lambda x: remove_stopwords(x, stop_words=stop_words))

In [40]:
df['merged_ingredients_wo_sw_lem'] = df['merged_ingredients_wo_sw'].apply(lemmatize_text)


In [19]:
def count_vectorize(
    df:pd.DataFrame,
    column_name:str,
    min_df:float=1.0,
    max_df:float=1.0,
    max_features:int=None,
    ngram_range=(1,1)
    )->pd.DataFrame:
    """
    usage :
    CountVectorize the text in column column_name and add to the Data Frame the new columns

    Args:
        df (pd.DataFrame): DataFrame in which there is a preprocessed text column to be vectorized
        colum_name (str) : name of the column that contains the text to CountVectorize

    Returns:
        pd.DataFrame

    """

    count_vectorizer = CountVectorizer(
        min_df=min_df,
        max_df=max_df,
        max_features=max_features,
        ngram_range=ngram_range
        )

    X = count_vectorizer.fit_transform(df[column_name])
    X_df = pd.DataFrame(X.toarray(), columns = count_vectorizer.get_feature_names_out())

    return pd.concat([df, X_df], axis = 1)


In [13]:

df2 = preprocessing.count_vectorize(
    df,
    'merged_steps',
    min_df=0.1,
    max_df=0.95
    )
df2

,name,minutes,contributor_id,submitted,tags,n_steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,add,apple,aside,bake,baking,beat,boil,bottom,bowl,bring,brown,butter,cheese,cinnamon,combine,cook,cooking,cool,cover,cream,cup,cut,degree,dish,drain,dry,egg,flour,garlic,golden,half,heat,high,...,low,make,medium,milk,minute,mix,mixture,oil,one,onion,oven,pan,pepper,place,pour,powder,preheat,put,rack,remaining,remove,salt,sauce,saucepan,serve,serving,set,sheet,side,simmer,skillet,slice,small,smooth,spoon,spread,sprinkle,stir,stirring,sugar,tender,time,together,top,turn,using,vanilla,warm,water,well
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,minute le course main ingredient cuisine prepa...,11,autumn favorite time year cook recipe prepared...,make choice proceed recipe depending size squa...,7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,make choice proceed recipe depending size squa...,0,0,0,1,1,0,0,0,0,0,0,3,0,0,0,0,0,0,1,0,0,3,1,0,0,0,0,0,0,0,2,0,0,...,0,1,0,0,1,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
31490,bit different breakfast pizza,30,26278,2002-06-17,minute le course main ingredient cuisine prepa...,9,recipe call crust prebaked bit adding ingredie...,preheat oven degree f press dough bottom side ...,6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,preheat oven degree f press dough bottom side ...,0,0,0,2,0,0,0,1,1,0,1,0,2,0,0,0,0,0,0,0,0,1,1,0,0,0,3,0,0,0,0,0,0,...,0,0,0,1,2,0,1,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
112140,kitchen chili,130,196586,2005-02-25,course preparation main dish chili crock pot s...,6,modified version mom chili hit christmas party...,brown ground beef large pot add chopped onion ...,13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,brown ground beef large pot add chopped onion ...,3,0,0,0,0,0,0,0,0,0,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59389,alouette potato,45,68585,2003-04-14,minute le course main ingredient preparation o...,11,super easy great tasting make ahead side dish ...,place potato large pot lightly salted water br...,11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,place potato large pot lightly salted water br...,1,0,1,1,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0,1,0,1,2,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1
44061,amish tomato ketchup canning,190,41706,2002-10-25,weeknight course main ingredient cuisine prepa...,5,dh amish mother raised recipe much prefers sto...,mix ingredient boil hour thick pour jar use ol...,8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,mix ingredient boil hour thick pour jar use ol...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81712,apricot barbecued chicken,45,67899,2004-01-21,minute le course main ingredient preparation o...,6,look scrumptious sweet wait try jan feb issue ...,large skillet cook chicken garlic oil browned ...,8,266.6,7.0,61.0,9.0,53.0,4.0,9.0,large skillet cook chicken garlic oil browned ...,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,2,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
107482,apricot barley cass

In [10]:



def tfidf_vectorize(
    df:pd.DataFrame,
    column_name:str,
    min_df:float=1.0,
    max_df:float=1.0,
    max_features:int=None,
    ngram_range=(1,1))->pd.DataFrame:
    """
    usage :
    TFIDF Vectorize the text in column column_name and add to the Data Frame the new columns

    Args:
        df (pd.DataFrame): DataFrame in which there is a preprocessed text column to be vectorized
        colum_name (str) : name of the column that contains the text to Vectorize

    Returns:
        pd.DataFrame

    """
    tf_idf_vectorizer = TfidfVectorizer(
        min_df=min_df,
        max_df=max_df,
        max_features=max_features,
        ngram_range=ngram_range
        )

    X = tf_idf_vectorizer.fit_transform(df[column_name])
    X_df = pd.DataFrame(X.toarray(), columns = tf_idf_vectorizer.get_feature_names_out())

    return pd.concat([df, X_df], axis = 1)

In [57]:
df = tfidf_vectorize(df, 'merged_ingredients_wo_sw_lem')
df

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,merged_steps_length,merged_ingredients,merged_tags,merged_ingredients_wo_sw,merged_ingredients_wo_sw_lem,10,100,10x,12,13,15,151,16,18,21,35,43,57,70,80,85,90,92,93,94,95,96,97,98,99,abalone,...,worthington,wrap,wrapper,wraps,wyler,xanthan,xxxx,xylitol,yabbies,yacon,yak,yakisoba,yam,yams,yard,yautia,yeast,yellow,yellowfin,yellowtail,yigandes,yoghurt,yogurt,yolk,yolks,yoplait,york,yoshida,young,yucca,yukon,yum,yuzu,yve,yvette,za,zatarain,zatarains,zatarians,zeera,zero,zest,zesty,zinfandel,zinger,ziploc,ziti,zoom,zucchini,zwieback
137739,arriba baked winter squash mexican style,55.0,47892.0,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11.0,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7.0,51.5,0.0,13.0,0.0,2.0,0.0,4.0,make a choice and proceed with recipe\ndependi...,697.0,winter squash\nmexican seasoning\nmixed spice\...,60-minutes-or-less\ntime-to-make\ncourse\nmain...,winter squash mexican seasoning mixed spice ho...,winter squash mexican seasoning mixed spice ho...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31490,a bit different breakfast pizza,30.0,26278.0,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9.0,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6.0,173.4,18.0,0.0,17.0,22.0,35.0,1.0,preheat oven to 425 degrees f\npress dough int...,386.0,prepared pizza crust\nsausage patty\neggs\nmil...,30-minutes-or-less\ntime-to-make\ncourse\nmain...,prepared pizza crust sausage patty eggs milk s...,prepared pizza crust sausage patty egg milk sa...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112140,all in the kitchen chili,130.0,196586.0,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6.0,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13.0,269.8,22.0,32.0,48.0,39.0,27.0,5.0,brown ground beef in large pot\nadd chopped on...,293.0,ground beef\nyellow onions\ndiced tomatoes\nto...,time-to-make\ncourse\npreparation\nmain-dish\n...,ground beef yellow onions diced tomatoes tomat...,ground beef yellow onion dice tomato tomato pa...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.274827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59389,alouette potatoes,45.0,68585.0,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11.0,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",11.0,368.1,17.0,10.0,2.0,14.0,8.0,20.0,place potatoes in a large pot of lightly salte...,543.0,spreadable cheese with garlic and herbs\nnew p...,60-minutes-or-less\n

In [13]:
my_data.iloc[231635]

name                         cookies by design   cookies on a stick
id                                                           298512
minutes                                                          29
contributor_id                                               506822
submitted                                       2008-04-15 00:00:00
tags              ['30-minutes-or-less', 'time-to-make', 'course...
nutrition                 [188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]
n_steps                                                           9
steps             ['place melted butter in a large mixing bowl a...
description       i've heard of the 'cookies by design' company,...
ingredients       ['butter', 'eagle brand condensed milk', 'ligh...
n_ingredients                                                    10
Name: 231635, dtype: object